## pybind11 array and buffer interface: A numpy user’s perspective

![pybind11-logo](images/pybind11_logo.png)

Dboy Liao

[GitHub](https://github.com/dboyliao)
[LinkedIn](https://www.linkedin.com/in/yin-chen-liao-69967188/)
[CakeResume](https://www.cakeresume.com/dboyliao)

# The Numpy `ndarray`

In [1]:
import numpy as np

In [2]:
array = np.arange(16, dtype=np.int8).reshape(4, 4).copy()
array

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]], dtype=int8)

```python
array.shape
```

```
array.strides
```

```python
array.data
```

```python
array.base
```

In [3]:
np.lib.stride_tricks

<module 'numpy.lib.stride_tricks' from '/Users/dboyliao/.pyenv/versions/3.8.1/lib/python3.8/site-packages/numpy/lib/stride_tricks.py'>

![array-2d](images/array_2d.drawio.svg)

In [4]:
print(array)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


![array-2d-flatten](images/array_2d_flatten.drawio.svg)

In [5]:
arr_flatten = array.ravel()
arr_flatten

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
      dtype=int8)

In [6]:
print("1 == 4 * 0 + 1:", 1 == 4 * 0 + 1)
print("6 == 4 * 1 + 2:", 6 == 4 * 1 + 2)

1 == 4 * 0 + 1: True
6 == 4 * 1 + 2: True


In [7]:
print("arr_flatten[1] == array[0, 1]:", arr_flatten[1] == array[0, 1])
print("arr_flatten[6] == array[1, 2]:", arr_flatten[6] == array[1, 2])

arr_flatten[1] == array[0, 1]: True
arr_flatten[6] == array[1, 2]: True


In [8]:
array.strides

(4, 1)

- linear offset: the offset of an element in the flattened array
- $\mathbf{arr}$: a m-dims array
    - strides: $(s_0, s_1, ..., s_m)$
    - shape: $(d_0, d_1, ..., d_m)$
- $e = \mathbf{arr}[i_0, i_1, ..., i_m]$, element in $\mathbf{arr}$
    - with linear offset $\text{offset}_e$

Then we have:

$$
    \text{offset}_e = \sum\limits_{j=0}^{m} d_j \cdot s_j
$$

![array-2d-view](images/array_2d_view.drawio.svg)

In [9]:
array.reshape(2, 2, 2, 2)

array([[[[ 0,  1],
         [ 2,  3]],

        [[ 4,  5],
         [ 6,  7]]],


       [[[ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15]]]], dtype=int8)

In [10]:
array.reshape(2, 2, 2, 2).ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
      dtype=int8)

![okey-then-what](images/okey_then_what.jpg)

## Broadcasting Rule: From Scratch

[Numpy Documentation](https://numpy.org/doc/stable/user/basics.broadcasting.html#general-broadcasting-rules)

In [11]:
arr1 = np.arange(16, dtype=np.int8).reshape(4, 2, 2)
arr2 = np.array([1, 2], dtype=np.int8)

In [12]:
arr1.shape

(4, 2, 2)

In [13]:
arr2.shape

(2,)

In [14]:
arr2.strides

(1,)

In [15]:
# promote arr2: make it of the shape as arr1
arr2_promoted = np.lib.stride_tricks.as_strided(
    arr2,
    shape=arr1.shape,
    strides=(0, 0)+arr2.strides
)

In [16]:
arr2_promoted.shape

(4, 2, 2)

In [17]:
arr2_promoted

array([[[1, 2],
        [1, 2]],

       [[1, 2],
        [1, 2]],

       [[1, 2],
        [1, 2]],

       [[1, 2],
        [1, 2]]], dtype=int8)

In [18]:
np.alltrue(arr1 + arr2 == arr1 + arr2_promoted)

True

In [19]:
arr1_promoted_lib, arr2_promoted_lib = np.lib.stride_tricks.broadcast_arrays(arr1, arr2)

In [20]:
arr2_promoted_lib.strides

(0, 0, 1)

In [21]:
# arr2 is the owner of the underlying memory block
arr2.base is None

True

In [22]:
# arr2_promoted_lib is just a shared-memory view of arr2
arr2_promoted_lib.base is arr2

True

In [23]:
array.flatten().base is None

True

In [24]:
array.ravel().base is None

False

![okey-then-what](images/okey_then_what.jpg)

## Time Series: Sliding Window with Shared-Memory View

In [25]:
data = np.arange(20, dtype=np.int8)
data

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19], dtype=int8)

In [26]:
windows = np.lib.stride_tricks.as_strided(
    data,
    shape=(16, 5),
    strides=(1, 1)
)

In [27]:
windows

array([[ 0,  1,  2,  3,  4],
       [ 1,  2,  3,  4,  5],
       [ 2,  3,  4,  5,  6],
       [ 3,  4,  5,  6,  7],
       [ 4,  5,  6,  7,  8],
       [ 5,  6,  7,  8,  9],
       [ 6,  7,  8,  9, 10],
       [ 7,  8,  9, 10, 11],
       [ 8,  9, 10, 11, 12],
       [ 9, 10, 11, 12, 13],
       [10, 11, 12, 13, 14],
       [11, 12, 13, 14, 15],
       [12, 13, 14, 15, 16],
       [13, 14, 15, 16, 17],
       [14, 15, 16, 17, 18],
       [15, 16, 17, 18, 19]], dtype=int8)

In [28]:
X = windows[:, :4]
Y = windows[:, 4]

In [29]:
X

array([[ 0,  1,  2,  3],
       [ 1,  2,  3,  4],
       [ 2,  3,  4,  5],
       [ 3,  4,  5,  6],
       [ 4,  5,  6,  7],
       [ 5,  6,  7,  8],
       [ 6,  7,  8,  9],
       [ 7,  8,  9, 10],
       [ 8,  9, 10, 11],
       [ 9, 10, 11, 12],
       [10, 11, 12, 13],
       [11, 12, 13, 14],
       [12, 13, 14, 15],
       [13, 14, 15, 16],
       [14, 15, 16, 17],
       [15, 16, 17, 18]], dtype=int8)

In [30]:
Y

array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
      dtype=int8)

In [31]:
X.base is None

False

In [32]:
Y.base is None

False

In [33]:
X2 = windows[:, [0, 1, 2, 3]]

In [34]:
np.alltrue(X2 == X)

True

In [35]:
X2.base is None

False

- `start:end:step` is a short-hand for `slice` object
    - has **FIXED** pattern (fixed start, fixed end, fixed step)
    - can be convert to a set of corresponding strides
    - shared-memory view is possible
    - **NO COPY REQUIRED**

- a list of indices
    - no guarantee on the pattern of the indices contained
    - cannot be convert to a set of corresponding strides
    - **COPY IS A MUST**

# `pybind11` Buffer and Array Interface